In [1]:
from __future__ import print_function

In [7]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [8]:
batch_size = 64
train_dataset = datasets.MNIST(root='./data/mnist_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='./data/mnist_data/',
                             train=False,
                             transform=transforms.ToTensor())
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv2BN = nn.BatchNorm2d(10)
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.conv2BN(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [13]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [14]:
for epoch in range(1, 10):
    train(epoch)
    test()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300894
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.110224
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.945562
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.617471
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.121930
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.936609
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.605112
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.683228
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.545685
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.458210
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.317359
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.362346
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.334422
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.370502
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.186973
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.453349
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.338802
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.329893
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.213348
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.330546
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.1047, Accuracy: 9663/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.155291
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.091552
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.075712
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.141456
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.028995
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.311958
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.140763
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.036006
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.102197
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.055728
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.128585
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.189587
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.112383
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.082070
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.027378
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.032527
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.084622
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.122488
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.042914
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.097497
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.023315
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.061972
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.017953
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.135349
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.059926
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.257230
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.095183
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.052271
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.060287
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.017698
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.042986
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.053715
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.063886
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.018400
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.025605
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.076673
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.026815
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.059224
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.069627


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.074606
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.045577
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.058621
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.075079
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.174756
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.030500
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.018114
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.099067
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.024031
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.055943
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.089429
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.033116
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.030832
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.012425
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.048916
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.074094
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.019946
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.073076
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.031138
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.011217


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.205079
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.027639
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.071171
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.020705
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.037365
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.003038
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.014057
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.004208
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.016417
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.006418
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.069974
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.004649
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.004003
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.080953
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.043900
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.022385
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.035816
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.040950
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.090942
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.100971


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.026612
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.006386
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.036768
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.036335
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.060286

Test set: Average loss: 0.0360, Accuracy: 9876/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.030944
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.013190
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.015350
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.037442
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.044862
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.034661
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.007572
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.080062
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.013283
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.065326
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.017057
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.012797
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.024008
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.063482
Train Epoch: 9 [